# Introduction


This notebook demonstrates how BioThings Explorer can be used to execute queries having more than one intermediate nodes (here a "three-hop query").  **Note that this notebook can also be run and modified in Google CoLab ([link](https://colab.research.google.com/github/biothings/biothings_explorer/blob/master/jupyter%20notebooks/Disease%20-%20Gene%20-%20Disease%20-%20Chem%202021-02-03%20relay.ipynb)).**

The original posed TRAPI query was this:

```
{
 "message": {
  "query_graph": {
    "edges": {
      "e0": {
        "predicate": "biolink:gene_associated_with_condition",
        "subject": "n0",
        "object": "n1"
      },
      "e1": {
        "predicate":     "biolink:gene_has_variant_that_contributes_to_disease_association",
        "subject": "n0",
        "object": "n2"
      },
      "e2": {
        "subject": "n2",
        "object": "n3",
        "predicate": "biolink:correlated_with"
      }
    },
    "nodes": {
      "n0": {
        "category": "biolink:Gene"
      },
      "n1": {
        "category": "biolink:Disease",
        "id": "MONDO:0007254"
      },
      "n2": {
        "category": "biolink:Disease"
      },
      "n3": {
        "category": "biolink:ChemicalSubstance"
      }
    }
  }
 }
}
```

The original query starts with **"breast cancer" (MONDO:0007254)**.  To reduce the combinatorial explosion, I've replaced that with **"NGLY1-deficiency" (MONDO:0014109)** in the notebook below. The rest of the query path joins that initial disease to a `ChemicalSubstance`, joined by two intermediate nodes with type `Gene` and `Disease`.


**Background**: BioThings Explorer can answer two classes of queries -- "EXPLAIN" and "PREDICT".  EXPLAIN queries are described in [EXPLAIN_demo.ipynb](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/EXPLAIN_demo.ipynb), and PREDICT queries are described in [PREDICT_demo.ipynb](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/PREDICT_demo.ipynb). Here, we describe PREDICT queries and how to use BioThings Explorer to execute them.  A more detailed overview of the BioThings Explorer systems is provided in [these slides](https://docs.google.com/presentation/d/1QWQqqQhPD_pzKryh6Wijm4YQswv8pAjleVORCPyJyDE/edit?usp=sharing).

## Step 0: Load BioThings Explorer modules

Install the `biothings_explorer` and `biothings_schema` packages, as described in this [README](https://github.com/biothings/biothings_explorer/blob/master/jupyter%20notebooks/README.md#prerequisite).  This only needs to be done once (but including it here for compability with [colab](https://colab.research.google.com/)).

In [1]:
!pip install git+https://github.com/biothings/biothings_explorer#egg=biothings_explorer

Next, import the relevant modules:

* **Hint**: Find corresponding bio-entity representation used in BioThings Explorer based on user input (could be any database IDs, symbols, names)
* **FindConnection**: Find intermediate bio-entities which connects user specified input and output

In [2]:
from biothings_explorer.hint import Hint
from biothings_explorer.user_query_dispatcher import FindConnection
import nest_asyncio
nest_asyncio.apply()

## Step 1: Find representation of the input disease in BTE

In this step, BioThings Explorer translates our query string into a BioThings object, which contain mappings to many common identifiers.  Generally, the top result returned by the `Hint` module will be the correct item, but you should confirm that using the identifiers shown.

Search terms can correspond to any child of [BiologicalEntity](https://biolink.github.io/biolink-model/docs/BiologicalEntity.html) from the [Biolink Model](https://biolink.github.io/biolink-model/docs/), including `DiseaseOrPhenotypicFeature` (e.g., "lupus"), `ChemicalSubstance` (e.g., "acetaminophen"), `Gene` (e.g., "CDK2"), `BiologicalProcess` (e.g., "T cell differentiation"), and `Pathway` (e.g., "Citric acid cycle").

In [3]:
ht = Hint()
input_disease = ht.query("MONDO:0007254")["Disease"][0]

input_disease

{'MONDO': 'MONDO:0007254',
 'DOID': 'DOID:1612',
 'name': 'breast cancer',
 'MESH': 'D001943',
 'primary': {'identifier': 'MONDO',
  'cls': 'Disease',
  'value': 'MONDO:0007254'},
 'display': 'MONDO(MONDO:0007254) DOID(DOID:1612) MESH(D001943) name(breast cancer)',
 'type': 'Disease'}

## Step 2: Find `ChemicalSubstance`s that are associated with the input disease through `Gene` and `Disease` as intermediate nodes

In this section, we find all paths in the knowledge graph that connect the input disease to any entity that is a `ChemicalSubstance` feature.  To do that, we will use `FindConnection`.  This class is a convenient wrapper around two advanced functions for **query path planning** and **query path execution**. More advanced features for both query path planning and query path execution are in development and will be documented in the coming months. 

In [4]:
fc = FindConnection(input_obj=input_disease, 
                    output_obj='ChemicalSubstance', 
                    intermediate_nodes=['Gene', 'Disease'])

In [5]:
fc.connect(verbose=True)

========== QUERY PARAMETER SUMMARY ==========

BTE will find paths that join 'breast cancer' and 'ChemicalSubstance'. Paths will have 2 intermediate node.

Intermediate node #1 will have these type constraints: Gene

Intermediate node #2 will have these type constraints: Disease



========== QUERY #1 -- fetch all Gene entities linked to breast cancer ==========

==== Step #1: Query path planning ====

Because breast cancer is of type 'Disease', BTE will query our meta-KG for APIs that can take 'Disease' as input and 'Gene' as output

BTE found 8 apis:

API 1. hetio(1 API call)
API 2. mgi_gene2phenotype(1 API call)
API 3. pharos(1 API call)
API 4. DISEASES(1 API call)
API 5. biolink(1 API call)
API 6. cord_disease(1 API call)
API 7. scigraph(1 API call)
API 8. scibite(1 API call)


==== Step #2: Query path execution ====
NOTE: API requests are dispatched in parallel, so the list of APIs below is ordered by query time.

API 5.1: https://api.monarchinitiative.org/api/bioentity/disease/MO

API 9.15: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=25992
API 9.15 ctd failed
API 8.34: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:29086/diseases?rows=200&direct=true
API 9.22: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=387
API 9.22 ctd failed
API 8.82: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:400619/diseases?rows=200&direct=true
API 9.24: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=7517
API 9.24 ctd failed
API 9.23: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=6663
API 9.23 ctd failed
API 8.70: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3417/diseases?rows=200&direct=true
API 8.36: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:10370/diseases?rows=200&direct=true
API 8.96: https://api.mo

API 9.56: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=150353
API 9.56 ctd failed
API 9.58: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=3957
API 9.58 ctd failed
API 11.55: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:81624
API 8.112: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:57801/diseases?rows=200&direct=true
API 9.57: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=4907
API 9.59: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=23038
API 9.59 ctd failed
API 9.60: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=25758
API 9.60 ctd failed
API 9.57 ctd failed
API 11.57: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:4907
API 11.56: https://automat.renci.org/cord19-scib

API 9.105: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=79068
API 9.105 ctd failed
API 8.133: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3082/diseases?rows=200&direct=true
API 9.106: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=25996
API 9.106 ctd failed
API 8.131: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:3990/diseases?rows=200&direct=true
API 11.88: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:5228
API 9.107: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=2925
API 9.107 ctd failed
API 11.89: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:6581
API 9.108: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=6781
API 9.108 ctd failed
API 8.135: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene

API 9.134: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=84436
API 9.134 ctd failed
API 11.115: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:1730
API 8.153: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:9048/diseases?rows=200&direct=true
API 11.114: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:170589
API 9.135: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=54890
API 9.135 ctd failed
API 8.147: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6997/diseases?rows=200&direct=true
API 9.137: http://ctdbase.org/tools/batchQuery.go?inputType=gene&report=diseases_curated&format=json&inputTerms=1312
API 9.137 ctd failed
API 11.116: https://automat.renci.org/cord19-scibite/gene/disease/NCBIGene:8314
API 8.159: https://api.monarchinitiative.org/api/bioentity/gene/NCBIGene:6275/diseases?rows=200&direct=true
API 9.136: http://

...

API 7.1470 scigraph: No hits
API 7.1471 scigraph: No hits
API 7.1472 scigraph: No hits
API 7.1473 scigraph: No hits
API 7.1474 scigraph: No hits
API 7.1475 scigraph: No hits
API 7.1476 scigraph: No hits
API 7.1477 scigraph: No hits
API 7.1478 scigraph: No hits
API 7.1479 scigraph: No hits
API 7.1480 scigraph: No hits
API 7.1481 scigraph: No hits
API 7.1482 scigraph: No hits
API 7.1483 scigraph: No hits
API 7.1484 scigraph: No hits
API 7.1485 scigraph: No hits
API 7.1486 scigraph: No hits
API 7.1487 scigraph: No hits
API 7.1488 scigraph: No hits
API 7.1489 scigraph: No hits
API 7.1490 scigraph: No hits
API 7.1491 scigraph: No hits
API 7.1492 scigraph: No hits
API 7.1493 scigraph: No hits
API 7.1494 scigraph: No hits
API 7.1495 scigraph: No hits
API 7.1496 scigraph: No hits
API 7.1497 scigraph: No hits
API 7.1498 scigraph: No hits
API 7.1499 scigraph: No hits
API 7.1500 scigraph: No hits
API 7.1501 scigraph: No hits
API 7.1502 scigraph: No hits
API 7.1503 scigraph: No hits
API 7.1504 sci

API 8.656 scibite: No hits
API 8.657 scibite: No hits
API 8.658 scibite: No hits
API 8.659 scibite: No hits
API 8.660 scibite: No hits
API 8.661 scibite: No hits
API 8.662 scibite: No hits
API 8.663 scibite: No hits
API 8.664 scibite: No hits
API 8.665 scibite: No hits
API 8.666 scibite: No hits
API 8.667 scibite: No hits
API 8.668 scibite: No hits
API 8.669 scibite: No hits
API 8.670 scibite: No hits
API 8.671 scibite: No hits
API 8.672 scibite: No hits
API 8.673 scibite: No hits
API 8.674 scibite: No hits
API 8.675 scibite: No hits
API 8.676 scibite: No hits
API 8.677 scibite: No hits
API 8.678 scibite: No hits
API 8.679 scibite: No hits
API 8.680 scibite: No hits
API 8.681 scibite: No hits
API 8.682 scibite: No hits
API 8.683 scibite: No hits
API 8.684 scibite: No hits
API 8.685 scibite: No hits
API 8.686 scibite: No hits
API 8.687 scibite: No hits
API 8.688 scibite: No hits
API 8.689 scibite: No hits
API 8.690 scibite: No hits
API 8.691 scibite: No hits
API 8.692 scibite: No hits
A

API 8.1893 scibite: No hits
API 8.1894 scibite: No hits
API 8.1895 scibite: No hits
API 8.1896 scibite: No hits
API 8.1897 scibite: No hits
API 8.1898 scibite: No hits
API 8.1899 scibite: No hits
API 8.1900 scibite: No hits
API 8.1901 scibite: No hits
API 8.1902 scibite: No hits
API 8.1903 scibite: No hits
API 8.1904 scibite: No hits
API 8.1905 scibite: No hits
API 8.1906 scibite: No hits
API 8.1907 scibite: No hits
API 8.1908 scibite: No hits
API 8.1909 scibite: No hits
API 8.1910 scibite: No hits
API 8.1911 scibite: No hits
API 8.1912 scibite: No hits
API 8.1913 scibite: No hits
API 8.1914 scibite: No hits
API 8.1915 scibite: No hits
API 8.1916 scibite: No hits
API 8.1917 scibite: No hits
API 8.1918 scibite: No hits
API 8.1919 scibite: No hits
API 8.1920 scibite: No hits
API 8.1921 scibite: No hits
API 8.1922 scibite: No hits
API 8.1923 scibite: No hits
API 8.1924 scibite: No hits
API 8.1925 scibite: No hits
API 8.1926 scibite: No hits
API 8.1927 scibite: No hits
API 8.1928 scibite: 

API 3.1320 pharos: No hits
API 3.1321 pharos: No hits
API 3.1322 pharos: No hits
API 3.1323 pharos: No hits
API 3.1324 pharos: No hits
API 3.1325 pharos: No hits
API 3.1326 pharos: No hits
API 3.1327 pharos: No hits
API 3.1328 pharos: No hits
API 3.1329 pharos: No hits
API 3.1330 pharos: No hits
API 3.1331 pharos: No hits
API 3.1332 pharos: No hits
API 3.1333 pharos: No hits
API 3.1334 pharos: No hits
API 3.1335 pharos: No hits
API 3.1336 pharos: No hits
API 3.1337 pharos: No hits
API 3.1338 pharos: No hits
API 3.1339 pharos: No hits
API 3.1340 pharos: No hits
API 3.1341 pharos: No hits
API 3.1342 pharos: No hits
API 3.1343 pharos: No hits
API 3.1344 pharos: No hits
API 3.1345 pharos: No hits
API 3.1346 pharos: No hits
API 3.1347 pharos: No hits
API 3.1348 pharos: No hits
API 3.1349 pharos: No hits
API 3.1350 pharos: No hits
API 3.1351 pharos: No hits
API 3.1352 pharos: No hits
API 3.1353 pharos: No hits
API 3.1354 pharos: No hits
API 3.1355 pharos: No hits
API 3.1356 pharos: No hits
A


After id-to-object translation, BTE retrieved 3174 unique objects.

========== Final assembly of results ==========


In the #1 query, BTE found 1653 unique Gene nodes
In the #2 query, BTE found 2048 unique Disease nodes
In the #3 query, BTE found 3174 unique ChemicalSubstance nodes


## Step 3: Assemble and filter the results

Let's create a `df` object that contains the full output from BioThings Explorer. Each row will show one path that joins the input node to an intermediate node (a `Gene`) to another intermediate node (a `Disease`) to an ending node (a `ChemicalSubstance`). The data frame includes a set of columns with additional details on each node and edge (including human-readable labels, identifiers, and sources). 

In [6]:
df = fc.display_table_view()

In [7]:
df

input input_type       pred1                     pred1_source  \
0       BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
1       BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
2       BREAST CANCER    Disease  related_to                          DISEASE   
3       BREAST CANCER    Disease  related_to                          DISEASE   
4       BREAST CANCER    Disease  related_to                          DISEASE   
...               ...        ...         ...                              ...   
297429  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
297430  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
297431  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
297432  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
297433  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   

               pred1_api pred1_pubmed node1_type node1_name         node1_id  \
0       CORD Disease API         None       Gene      CDH17    NCBIGene:1015   
1       CORD Disease API         None       Gene       SNCA    NCBIGene:6622   
2           DISEASES API         None       Gene     BRMS1L   NCBIGene:84312   
3           DISEASES API         None       Gene     DAB2IP  NCBIGene:153090   
4           DISEASES API         None       Gene      PCSK9  NCBIGene:255738   
...                  ...          ...        ...        ...              ...   
297429  CORD Disease API         None       Gene        CFH    NCBIGene:3075   
297430  CORD Disease API         None       Gene        CFH    NCBIGene:3075   
297431  CORD Disease API         None       Gene        CFH    NCBIGene:3075   
297432  CORD Disease API         None       Gene        CFH    NCBIGene:3075   
297433  CORD Disease API         None       Gene        CFH    NCBIGene:3075   

             pred2  ... node2_type         node2_name             node2_id  \
0       related_to  ...    Disease                DMD  MONDO:MONDO:0010679   
1       related_to  ...    Disease                DMD  MONDO:MONDO:0010679   
2       related_to  ...    Disease                CAD  MONDO:MONDO:0005010   
3       related_to  ...    Disease                CAD  MONDO:MONDO:0005010   
4       related_to  ...    Disease                CAD  MONDO:MONDO:0005010   
...            ...  ...        ...                ...                  ...   
297429  related_to  ...    Disease  DISEASE OF KIDNEY  MONDO:MONDO:0005240   
297430  related_to  ...    Disease  DISEASE OF KIDNEY  MONDO:MONDO:0005240   
297431  related_to  ...    Disease  DISEASE OF KIDNEY  MONDO:MONDO:0005240   
297432  related_to  ...    Disease  DISEASE OF KIDNEY  MONDO:MONDO:0005240   
297433  related_to  ...    Disease  DISEASE OF KIDNEY  MONDO:MONDO:0005240   

                     pred3                     pred3_source         pred3_api  \
0               related_to  Translator Text Mining Provider  CORD Disease API   
1               related_to  Translator Text Mining Provider  CORD Disease API   
2               related_to  Translator Text Mining Provider  CORD Disease API   
3               related_to  Translator Text Mining Provider  CORD Disease API   
4               related_to  Translator Text Mining Provider  CORD Disease API   
...                    ...                              ...               ...   
297429  contraindicated_by                      drugcentral   MyChem.info API   
297430          related_to  Translator Text Mining Provider  CORD Disease API   
297431          related_to  Translator Text Mining Provider  CORD Disease API   
297432          related_to  Translator Text Mining Provider  CORD Disease API   
297433          related_to  Translator Text Mining Provider  CORD Disease API   

       pred3_pubmed        output_type  \
0               NaN  ChemicalSubstance   
1               NaN  ChemicalSubstance   
2               NaN  ChemicalSubstance   
3   

Let's do some profiling of the predicates, and then let's filter `pred3` to only include `treated_by` edges.

In [8]:
df.pred1.value_counts()

related_to    273471
Name: pred1, dtype: int64

In [9]:
df.pred2.value_counts()

related_to            273290
contraindicated_by       181
Name: pred2, dtype: int64

In [10]:
df.pred3.value_counts()

related_to            186164
contraindicated_by     63366
treated_by             23650
Name: pred3, dtype: int64

In [11]:
dfFilt = df.loc[df['output_name'].notnull()].query('pred3 == "treated_by"')

In [12]:
dfFilt

input input_type       pred1                     pred1_source  \
2293    BREAST CANCER    Disease  related_to                          DISEASE   
2295    BREAST CANCER    Disease  related_to                          DISEASE   
2296    BREAST CANCER    Disease  related_to                          DISEASE   
2297    BREAST CANCER    Disease  related_to                          DISEASE   
2298    BREAST CANCER    Disease  related_to                          DISEASE   
...               ...        ...         ...                              ...   
297296  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
297297  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
297298  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
297299  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
297312  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   

               pred1_api pred1_pubmed node1_type node1_name        node1_id  \
2293        DISEASES API         None       Gene      ERBB4   NCBIGene:2066   
2295        DISEASES API         None       Gene       ESR2   NCBIGene:2100   
2296        DISEASES API         None       Gene      ZFHX3    NCBIGene:463   
2297        DISEASES API         None       Gene     ZNF462  NCBIGene:58499   
2298        DISEASES API         None       Gene      AKAP6   NCBIGene:9472   
...                  ...          ...        ...        ...             ...   
297296  CORD Disease API         None       Gene       RGS5   NCBIGene:8490   
297297  CORD Disease API         None       Gene     CYP3A5   NCBIGene:1577   
297298  CORD Disease API         None       Gene       RGS5   NCBIGene:8490   
297299  CORD Disease API         None       Gene     CYP3A5   NCBIGene:1577   
297312  CORD Disease API         None       Gene        MSN   NCBIGene:4478   

             pred2  ... node2_type                     node2_name  \
2293    related_to  ...    Disease                          A-FIB   
2295    related_to  ...    Disease                          A-FIB   
2296    related_to  ...    Disease                          A-FIB   
2297    related_to  ...    Disease                          A-FIB   
2298    related_to  ...    Disease                          A-FIB   
...            ...  ...        ...                            ...   
297296  related_to  ...    Disease                            EHT   
297297  related_to  ...    Disease                            EHT   
297298  related_to  ...    Disease                            EHT   
297299  related_to  ...    Disease                            EHT   
297312  related_to  ...    Disease  ACUTE GASTRIC MUCOSAL EROSION   

                   node2_id       pred3 pred3_source        pred3_api  \
2293    MONDO:MONDO:0004981  treated_by  drugcentral  MyChem.info API   
2295    MONDO:MONDO:0004981  treated_by  drugcentral  MyChem.info API   
2296    MONDO:MONDO:0004981  treated_by  drugcentral  MyChem.info API   
2297    MONDO:MONDO:0004981  treated_by  drugcentral  MyChem.info API   
2298    MONDO:MONDO:0004981  treated_by  drugcentral  MyChem.info API   
...                     ...         ...          ...              ...   
297296  MONDO:MONDO:0007781  treated_by  drugcentral  MyChem.info API   
297297  MONDO:MONDO:0007781  treated_by  drugcentral  MyChem.info API   
297298  MONDO:MONDO:0007781  treated_by  drugcentral  MyChem.info API   
297299  MONDO:MONDO:0007781  treated_by  drugcentral  MyChem.info API   
297312  MONDO:MONDO:0004966  treated_by  drugcentral  MyChem.info API   

       pred3_pubmed        output_type  \
2293            NaN  ChemicalSubstance   
2295            NaN  ChemicalSubstance   
2296            NaN  ChemicalSubstance   
2297            NaN  ChemicalSubstance   
2298            NaN  ChemicalSubstance   
...             ...                ...   
297296          NaN  ChemicalSubstance   
297297          NaN  ChemicalSubstance   
297298    

## Step 4: Summarize output `ChemicalSubstance`s

In [13]:
dfFilt.output_name.value_counts().head(50)

(11BETA)-11,17,21-TRIHYDROXYPREGNA-1,4-DIENE-3,20-DIONE                                                                                                                                                                                                                                            364
ADRENALEX                                                                                                                                                                                                                                                                                          361
(11BETA)-11,17,21-TRIHYDROXYPREGN-4-ENE-3,20-DIONE                                                                                                                                                                                                                                                 359
(11BETA,16ALPHA)-9-FLUORO-11,17,21-TRIHYDROXY-16-METHYLPREGNA-1,4-DIENE-3,20-DIONE                                 

Suppose we are interested in the links behind `BLENOXANE`. Let's examine that subset of paths, specifically printing the frequency of the intermediate `Gene`s and `Disease`s.

In [14]:
dfFilt2 = dfFilt.query('output_name == "BLENOXANE"')
dfFilt2

input input_type       pred1                     pred1_source  \
186796  BREAST CANCER    Disease  related_to               mgi_gene2phenotype   
186797  BREAST CANCER    Disease  related_to                          DISEASE   
186798  BREAST CANCER    Disease  related_to                          DISEASE   
186799  BREAST CANCER    Disease  related_to                          DISEASE   
186800  BREAST CANCER    Disease  related_to                          DISEASE   
186804  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
186805  BREAST CANCER    Disease  related_to                          DISEASE   
186806  BREAST CANCER    Disease  related_to                          DISEASE   
186807  BREAST CANCER    Disease  related_to                          DISEASE   
186808  BREAST CANCER    Disease  related_to                          DISEASE   
186809  BREAST CANCER    Disease  related_to                          DISEASE   
186810  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
186811  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
186812  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
186813  BREAST CANCER    Disease  related_to                          DISEASE   
186814  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   
186815  BREAST CANCER    Disease  related_to  Translator Text Mining Provider   

                    pred1_api pred1_pubmed node1_type node1_name  \
186796  MGIgene2phenotype API         None       Gene  MGI:98834   
186797           DISEASES API         None       Gene      ARAP3   
186798           DISEASES API         None       Gene        TYR   
186799           DISEASES API         None       Gene    SLC39A6   
186800           DISEASES API         None       Gene      BRCA2   
186804       CORD Disease API         None       Gene      BRCA2   
186805           DISEASES API         None       Gene      TRPS1   
186806           DISEASES API         None       Gene       MC1R   
186807           DISEASES API         None       Gene       RALY   
186808           DISEASES API         None       Gene      CSMD1   
186809           DISEASES API         None       Gene     DNAH11   
186810       CORD Disease API         None       Gene       SOX4   
186811       CORD Disease API         None       Gene      FOXP1   
186812       CORD Disease API         None       Gene        SRC   
186813           DISEASES API         None       Gene       BCL6   
186814       CORD Disease API         None       Gene       BCL2   
186815       CORD Disease API         None       Gene       RORA   

              node1_id       pred2  ... node2_type  \
186796   MGI:MGI:98834  related_to  ...    Disease   
186797  NCBIGene:64411  related_to  ...    Disease   
186798   NCBIGene:7299  related_to  ...    Disease   
186799  NCBIGene:25800  related_to  ...    Disease   
186800    NCBIGene:675  related_to  ...    Disease   
186804    NCBIGene:675  related_to  ...    Disease   
186805   NCBIGene:7227  related_to  ...    Disease   
186806   NCBIGene:4157  related_to  ...    Disease   
186807  NCBIGene:22913  related_to  ...    Disease   
186808  NCBIGene:64478  related_to  ...    Disease   
186809   NCBIGene:8701  related_to  ...    Disease   
186810   NCBIGene:6659  related_to  ...    Disease   
186811  NCBIGene:27086  related_to  ...    Disease   
186812   NCBIGene:6714  related_to  ...    Disease   
186813    NCBIGene:604  related_to  ...    Disease   
186814    NCBIGene:596  related_to  ...    Disease   
186815   NCBIGene:6095  related_to  ...    Disease   

                                 node2_name             node2_id       pred3  \
186796  ADULT DIFFUSE LARGE B-CELL LYMPHOMA  MONDO:MONDO:0018905  treated_by   
186797  ADULT DIFFUSE LARGE B-CELL LYMPHOMA  MONDO:MONDO:0018905  treated_by   
186798  CARCINOMA, SQUAMOUS CELL, MALIGNANT  MONDO:MONDO:0005096  treated_by   
186799  CARCINOMA, SQUAMOUS CELL, MALIG

In [15]:
dfFilt2.node1_name.value_counts().head(50)

BRCA2        2
BCL6         1
RALY         1
RORA         1
DNAH11       1
BCL2         1
MGI:98834    1
FOXP1        1
SLC39A6      1
TRPS1        1
TYR          1
MC1R         1
SRC          1
SOX4         1
ARAP3        1
CSMD1        1
Name: node1_name, dtype: int64

In [16]:
dfFilt2.node2_name.value_counts().head(50)

CARCINOMA, SQUAMOUS CELL, MALIGNANT    12
ADULT DIFFUSE LARGE B-CELL LYMPHOMA     2
FOLLICLE CENTER LYMPHOMA                2
CANCER OF CRANIOCERVICAL REGION         1
Name: node2_name, dtype: int64

## Step 5: export full results to TRAPI

(though this export is not yet TRAPI v1.0 compliant, pending resolution of [this issue](https://github.com/biothings/biothings_explorer/issues/169)...)

In [17]:
fc.to_reasoner_std()

{'query_graph': {'edges': [{'id': 'e0',
    'source_id': 'n0',
    'target_id': 'n1',
    'directed': True},
   {'id': 'e1', 'source_id': 'n1', 'target_id': 'n2', 'directed': True},
   {'id': 'e2', 'source_id': 'n2', 'target_id': 'n3', 'directed': True}],
  'nodes': [{'id': 'n0', 'type': 'Disease', 'curie': 'MONDO:0007254'},
   {'id': 'n1', 'type': 'Gene'},
   {'id': 'n2', 'type': 'Disease'},
   {'id': 'n3', 'type': 'ChemicalSubstance'}]},
 'knowledge_graph': {'nodes': [{'id': 'name:breast cancer',
    'name': 'BREAST CANCER',
    'type': 'Disease',
    'equivalent_identifiers': defaultdict(set,
                {'MONDO': ['MONDO:0007254'],
                 'DOID': ['DOID:1612'],
                 'name': ['BREAST CANCER',
                  'BREAST TUMOR',
                  'CANCER OF BREAST',
                  'MALIGNANT BREAST NEOPLASM',
                  'MALIGNANT BREAST TUMOR',
                  'MALIGNANT NEOPLASM OF BREAST',
                  'MALIGNANT NEOPLASM OF THE BREAST',
  